In [ ]:
# Initalize important stuff
import xspec as xs
import matplotlib.pyplot as plt, numpy as np, pandas as pd
import matplotlib.ticker as mticker
import seaborn as sns
sns.set(style='dark')
import time
from IPython.display import clear_output
def clear(): time.sleep(0.1), clear_output()

xs.AllModels.lmod("acx", "/home/vatsal/soft/atomdbcx/xspec")

In [ ]:
# Load original spectrum file
xs.AllData.clear()
original_data = xs.Spectrum('north_080420_src_bin10.ds')
xs.Plot.xAxis = "keV"
original_data.ignore("**-**")
original_data.notice("0.2-0.9")

xs.Plot('data')
xmm_df = pd.DataFrame({'x':xs.Plot.x(), 'y':xs.Plot.y(), 'xerr':xs.Plot.xErr(), 'yerr':xs.Plot.yErr()})

clear()

In [ ]:
# VACX MODEL
xs.AllModels.clear()
vacx = xs.Model('vacx')

vacx(1).values = 0.164071 		# plasma temp kT
vacx(2).values = 0.1 			# He
vacx(3).values = 0 			    # C
vacx(4).values = 0 			    # N
vacx(5).values = 0.297569 		# O
vacx(6).values = 0 			    # Ne
vacx(7).values = 0 			    # Mg
vacx(8).values = 0 			    # Al
vacx(9).values = 0 			    # Si
vacx(10).values= 8.23918 		# S
vacx(11).values= 0 			    # Ar
vacx(12).values= 0 			    # Ca
vacx(13).values= 0 			    # Fe
vacx(14).values= 0 			    # Ni
vacx(15).values= 0 			    # redshift
vacx(16).values= 0 			    # swcx
vacx(17).values= 5	 			# model
vacx(18).values= 1.40847e-06	# norm

vacx(2).frozen = True
vacx(5).frozen = False
vacx(10).frozen = False

xs.Fit.nIterations = 50
xs.Fit.perform()

xs.Plot('data')
vacx_folded = xs.Plot.model()

!rm vacx.xcm                     # Set name for model to be saved
xs.Xset.save('vacx.xcm')        # Set name for model to be saved
clear()

In [ ]:
# Faking in LEM
response_file = 'lem_09ev_110422.rmf'
arf_file      = 'lem_110422.arf'
exposure_time = '144000'

# VACX
xs.AllModels.clear()
xs.Xset.restore('vacx.xcm')     # Set name for model to be saved
!rm vacx.fak                # Set fake file name
xs.AllData.clear()
fs = xs.FakeitSettings(response=response_file, arf=arf_file, exposure=exposure_time)
fs.fileName = 'vacx.fak'    # Set fake file name
xs.AllData.fakeit(1,fs)
#xs.Fit.nIterations = 50
#xs.Fit.perform()
xs.Plot('data')
vacx_df = pd.DataFrame({'x':xs.Plot.x(), 'y':xs.Plot.y(), 'xerr':xs.Plot.xErr(), 'yerr':xs.Plot.yErr()})

clear()

In [ ]:
fig = plt.figure()

ax1 = fig.add_axes([0, -1.8, 1.2, 0.6], xlim=(0.25, 0.8))
ax1.plot(vacx_df.x, vacx_df.y)
ax1.grid(color='w')
ax1.set_ylabel('Counts / s / keV')
ax1.set_xlabel('Energy / keV')

# plt.savefig('acx-vacxCNO-vacxSO.pdf', bbox_inches='tight', dpi=600)

In [ ]:
# This is a bit of a lazy method of doing this but I felt this was way easier than any other method.
# I change the models (5-9) manually and run the code for each one, saving the final lem dataframes for each one of them.
# And then I compile them together using a different bit of code.


# Uncomment to save dataframe to csv
# lem_df.to_csv(r'm5_df.csv', index=False)

In [ ]:
# Compiling all the models togethre

m5_df = pd.read_csv('m5_df.csv')
m6_df = pd.read_csv('m6_df.csv')
m7_df = pd.read_csv('m7_df.csv')
m8_df = pd.read_csv('m8_df.csv')

In [ ]:
fig = plt.figure()

ax1 = fig.add_axes([0, 0, 1.2, 0.6], xlim=(0.25, 0.8), ylim=(-0.4,7.5))
ax1.plot(m5_df.x, m5_df.y, label='Evenly Weighted by Angular Momentum') 
ax1.grid(color='w'), ax1.set_xticklabels([]), ax1.set_ylabel('Counts / s / keV'), ax1.legend()

ax2 = fig.add_axes([0, -0.6, 1.2, 0.6],  xlim=(0.25, 0.8), ylim=(-0.4,7.5))
ax2.plot(m6_df.x, m6_df.y, label='Weighted by the Relative Statistical Weight of each level')
ax2.grid(color='w'), ax2.set_xticklabels([]), ax2.set_ylabel('Counts / s / keV'), ax2.legend()

ax3 = fig.add_axes([0, -1.2, 1.2, 0.6],  xlim=(0.25, 0.8), ylim=(-0.4,7.5))
ax3.plot(m7_df.x, m7_df.y, label='Landau-Zener Model') 
ax3.grid(color='w'), ax3.set_xticklabels([]), ax3.set_ylabel('Counts / s / keV'), ax3.legend()

ax4 = fig.add_axes([0, -1.8, 1.2, 0.6],  xlim=(0.25, 0.8), ylim=(-0.4,7.5))
ax4.plot(m8_df.x, m8_df.y, label='Separable Model')
ax4.grid(color='w'), ax4.set_ylabel('Counts / s / keV'), ax4.legend(), ax4.set_xlabel('Energy / keV')

ax1.set_title("Effect of Different CX Physics Models on LEM Spectrum")

# plt.savefig('different_models.pdf', bbox_inches='tight', dpi=600)